In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from nilearn import datasets, plotting
from nilearn.image import index_img
from nilearn.plotting import view_img
from nilearn.input_data import NiftiMasker

from bokeh.plotting import figure
from bokeh.models import HoverTool, Div
from bokeh.layouts import column
from bokeh.io import output_file, save

In [2]:
data = datasets.fetch_development_fmri(n_subjects=4)
subs_fmri_img = [nib.load(data.func[i]) for i in range(len(data.func))]
subs_fmri_data = [img.get_fdata() for img in subs_fmri_img]

[get_dataset_dir] Dataset found in /root/nilearn_data/development_fmri
[get_dataset_dir] Dataset found in /root/nilearn_data/development_fmri/development_fmri
[get_dataset_dir] Dataset found in /root/nilearn_data/development_fmri/development_fmri


In [3]:
all_flattened_data = [sub.flatten() for sub in subs_fmri_data]
combined_voxels = np.concatenate(all_flattened_data)
common_bins = np.linspace(combined_voxels.min(), combined_voxels.max(), 101)
plots_list = []
colors = ["dodgerblue", "orangered", "seagreen", "purple"]
title = Div(text="<h3>Voxel Value Distributions Across Individual Subjects</h3>",
            width=800, align='center')
for i, subject_voxels in enumerate(all_flattened_data):
    hist, edges = np.histogram(subject_voxels, bins=common_bins)
    fig_kwargs = dict(
        height=250,
        width=800,
        title=f"Distribution for Subject {i+1}",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        y_axis_type="log",
        outline_line_color=None,
        min_border_left=50
    )
    if i > 0:
        fig_kwargs['x_range'] = plots_list[0].x_range
    p = figure(**fig_kwargs)
    p.yaxis.axis_label = "Frequency"
    p.xaxis.axis_label = "Voxel Value"
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.quad(
        top=hist,
        bottom=0.1,
        left=edges[:-1],
        right=edges[1:],
        fill_color=colors[i % len(colors)],
        line_color="black",
        alpha=0.8
    )
    hover = HoverTool(tooltips=[("Range", "@left{0.00} to @right{0.00}"), ("Frequency", "@top")])
    p.add_tools(hover)
    plots_list.append(p)
layout = column(*plots_list, spacing=0)
final_layout = column(title, layout)
output_file("_a_voxel_value_distributions.html", title="_a_voxel_value_distributions.html")
save(final_layout)

'/workspaces/leclei_project/_a_voxel_value_distributions.html'

In [4]:
fig, axes = plt.subplots(2, 2, figsize=(7, 7))
slice_indices = [15, 20, 25, 30]
axes = axes.ravel()
for i, slice_index in enumerate(slice_indices):
    ax = axes[i]
    ax.imshow(subs_fmri_data[0][:, :, slice_index, 0], cmap='gray')
    ax.set_title(f"Slice at index {slice_index}")
    ax.axis('off')
plt.tight_layout()
fig.suptitle("Subject 1 Axial Plane fMRI Slices")
fig.subplots_adjust(top=0.9)
plt.savefig("_b_sub1_fmri_slices.png", dpi=300)
plt.close(fig)

In [5]:
interactive_raw_view = view_img(index_img(subs_fmri_img[0], 100), cmap='Greys', title="Subject 1 fMRI View")
interactive_raw_view.save_as_html("_c_sub1_interactive_fmri_viewer.html")
with open("_c_sub1_interactive_fmri_viewer.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Subject 1 fMRI View</title>",
    "<title>_c_sub1_interactive_fmri_viewer.html</title>"
)
with open("_c_sub1_interactive_fmri_viewer.html", "w") as file:
    file.write(content)

/usr/local/lib/python3.12/dist-packages/nilearn/plotting/html_stat_map.py:109: UserWarning: Threshold given was 1e-06, but the data has no values below -121.83067321777344. 
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [6]:
x, y, z = np.mgrid[-1:1:50j, -1:1:50j, -1:1:50j]
brain_data1 = np.swapaxes(subs_fmri_data[0][:,:,:,0], 0, 1)
brain_data2 = np.swapaxes(subs_fmri_data[0][:,:,:,100], 0, 1)
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'scene'}, {'type': 'scene'}],
           [{'type': 'scene', 'colspan': 2}, None]],
    subplot_titles=('Time Point 0', 'Time Point 100', 'Overlaid View')
)
fig.add_trace(go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=brain_data1.flatten(),
    isomin=-700, isomax=np.max(brain_data1),
    surface_count=1, colorscale='Greys', showscale=False,
    caps=dict(x_show=False, y_show=False), name='Time Point 0'
), row=1, col=1)
fig.add_trace(go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=brain_data2.flatten(),
    isomin=-700, isomax=np.max(brain_data2),
    surface_count=1, colorscale='Greys', showscale=False,
    caps=dict(x_show=False, y_show=False), name='Time Point 100'
), row=1, col=2)
fig.add_trace(go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=brain_data1.flatten(),
    isomin=-700, isomax=np.max(brain_data1),
    surface_count=1, colorscale='Blues', name='Time Point 0 (Overlay)',
    showscale=False, opacity=0.5, caps=dict(x_show=False, y_show=False)
), row=2, col=1)
fig.add_trace(go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=brain_data2.flatten(),
    isomin=-700, isomax=np.max(brain_data2),
    surface_count=1, colorscale='Reds', name='Time Point 100 (Overlay)',
    showscale=False, opacity=0.1, caps=dict(x_show=False, y_show=False)
), row=2, col=1)
axis_settings = dict(visible=False, showticklabels=False, showgrid=False, zeroline=False, title='')
fig.update_layout(
    title_text='Subject 1 fMRI 3D Isosurface',
    title_x=0.5,
    height=800, margin=dict(l=0, r=0, b=0, t=60),
    scene=dict(xaxis=axis_settings, yaxis=axis_settings, zaxis=axis_settings),
    scene2=dict(xaxis=axis_settings, yaxis=axis_settings, zaxis=axis_settings),
    scene3=dict(xaxis=axis_settings, yaxis=axis_settings, zaxis=axis_settings)
)
fig.write_html("_d_sub1_fmri_3d_isosurface.html")

In [7]:
variance_data = np.var(subs_fmri_data[0], axis=3)
variance_data_swapped = np.swapaxes(variance_data, 0, 1)
x, y, z = np.mgrid[-1:1:50j, -1:1:50j, -1:1:50j]
fig_variance = go.Figure(data=go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=variance_data_swapped.flatten(),
    isomin=np.percentile(variance_data_swapped[variance_data_swapped > 0], 90),
    isomax=np.max(variance_data_swapped),
    surface_count=200, colorscale='Viridis', colorbar_title_text='Variance',
    showscale=False, caps=dict(x_show=False, y_show=False)
))
fig_variance.update_layout(
    title_text='Subject 1 fMRI 3D Variance Over Time',
    scene=dict(
        xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)
fig_variance.write_html("_e_sub1_fmri_3d_variance.html")

In [8]:
masker = NiftiMasker()
masked_data = masker.fit_transform(data.func[0])
masked_data_with_confounds = masker.fit_transform(data.func[0], confounds=data.confounds)
confound_effect_signal = masked_data - masked_data_with_confounds
confounds_img = masker.inverse_transform(confound_effect_signal)

In [9]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()
for i, slice_index in enumerate(slice_indices):
    ax = axes[i]
    ax.imshow(confounds_img.get_fdata()[:, :, slice_index, 0], cmap='gray')
    ax.set_title(f"Slice at index {slice_index}")
    ax.axis('off')
plt.tight_layout()
fig.suptitle("Subject 1 Axial Plane Confound fMRI Slices")
fig.subplots_adjust(top=0.9)
plt.savefig("_f_sub1_confound_fmri_slices.png", dpi=300)
plt.close(fig)

In [10]:
interactive_confound_view = view_img(index_img(confounds_img, 0), cmap='Greys', title="Sub 1 Confound fMRI View")
interactive_confound_view.save_as_html("_g_sub1_interactive_confound_fmri_viewer.html")
with open("_g_sub1_interactive_confound_fmri_viewer.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Sub 1 Confound fMRI View</title>",
    "<title>_g_sub1_interactive_confound_fmri_viewer.html</title>"
)
with open("_g_sub1_interactive_confound_fmri_viewer.html", "w") as file:
    file.write(content)

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning:




In [11]:
masker = NiftiMasker()
masked_data = masker.fit_transform(data.func[0], confounds=data.confounds)
fmri_with_confound = masker.inverse_transform(masked_data)

In [12]:
interactive_confound_applied_view = view_img(index_img(fmri_with_confound, 0), cmap='Greys', title="Sub 1 fMRI w/ Confound View")
interactive_confound_applied_view.save_as_html("_h_sub1_interactive_confound_applied_fmri_viewer.html")
with open("_h_sub1_interactive_confound_applied_fmri_viewer.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Sub 1 fMRI w/ Confound View</title>",
    "<title>_h_sub1_interactive_confound_applied_fmri_viewer.html</title>"
)
with open("_h_sub1_interactive_confound_applied_fmri_viewer.html", "w") as file:
    file.write(content)

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning:




In [13]:
masker = NiftiMasker()
masked_data = masker.fit_transform(data.func[0], confounds=data.confounds)
thresholded_masked_data = masked_data * (masked_data > masked_data.mean())
thresholded_img = masker.inverse_transform(thresholded_masked_data)

In [14]:
interactive_confound_applied_thresh_view = view_img(index_img(thresholded_img, 0), cmap='Greys', title="Sub 1 fMRI w/ Confound Thresh View")
interactive_confound_applied_thresh_view.save_as_html("_i_sub1_interactive_confound_applied_thresh_fmri_viewer.html")
with open("_i_sub1_interactive_confound_applied_thresh_fmri_viewer.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Sub 1 fMRI w/ Confound Thresh View</title>",
    "<title>_i_sub1_interactive_confound_applied_thresh_fmri_viewer.html</title>"
)
with open("_i_sub1_interactive_confound_applied_thresh_fmri_viewer.html", "w") as file:
    file.write(content)

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning:




In [15]:
variance_data = np.var(thresholded_img.get_fdata(), axis=3)
variance_data_swapped = np.swapaxes(variance_data, 0, 1)
x, y, z = np.mgrid[-1:1:50j, -1:1:50j, -1:1:50j]
fig_variance = go.Figure(data=go.Isosurface(
    x=x.flatten(), y=y.flatten(), z=z.flatten(),
    value=variance_data_swapped.flatten(),
    isomin=np.percentile(variance_data_swapped[variance_data_swapped > 0], 90),
    isomax=np.max(variance_data_swapped),
    surface_count=200, colorscale='Viridis', colorbar_title_text='Variance',
    showscale=False, caps=dict(x_show=False, y_show=False)
))
fig_variance.update_layout(
    title_text='Subject 1 fMRI w/ Confound Thresholded 3D Variance Over Time',
    scene=dict(
        xaxis=dict(visible=False), yaxis=dict(visible=False), zaxis=dict(visible=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)
fig_variance.write_html("_j_sub1_fmri_confound_thresh_3d_variance.html")

In [16]:
atlas = datasets.fetch_atlas_schaefer_2018()
atlas_plot = view_img(atlas.maps, title="Schaefer 2018 Atlas")
atlas_plot.save_as_html("_m_schaefer_2018_atlas.html")
with open("_m_schaefer_2018_atlas.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Schaefer 2018 Atlas</title>",
    "<title>_m_schaefer_2018_atlas.html</title>"
)
with open("_m_schaefer_2018_atlas.html", "w") as file:
    file.write(content)

[get_dataset_dir] Dataset found in /root/nilearn_data/schaefer_2018


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning:




In [17]:
import pandas as pd
import h5py
import cv2
import ipywidgets as widgets
from IPython.display import Video, display
from moviepy import VideoFileClip
root_data_dir = '../algonauts_2025_challenge_tutorial_data'
def load_tsv_file(transcript_path):
    transcript_df = pd.read_csv(transcript_path, sep='\t')
    sample_transcript_data = transcript_df.iloc[:20]
    print("Transcript data (Rows 0 to 20):")
    display(sample_transcript_data)
    print(f"\nTranscript has {transcript_df.shape[0]} rows (chunks of 1.49 seconds) and {transcript_df.shape[1]} columns.")
def load_transcript(transcript_path):
    df = pd.read_csv(transcript_path, sep='\t')
    return df
def get_movie_info(movie_path):
    cap = cv2.VideoCapture(movie_path)
    fps, frame_count = cap.get(cv2.CAP_PROP_FPS), cap.get(cv2.CAP_PROP_FRAME_COUNT)
    cap.release()
    print(f"Movie FPS: {fps}, Frame Count: {frame_count}")
    return fps, frame_count / fps
def split_movie_into_chunks(movie_path, chunk_duration=1.49):
    _, video_duration = get_movie_info(movie_path)
    chunks = []
    start_time = 0.0
    while start_time < video_duration:
        end_time = min(start_time + chunk_duration, video_duration)
        chunks.append((start_time, end_time))
        start_time += chunk_duration
    return chunks
def extract_movie_segment_with_sound(movie_path, start_time, end_time,
    output_path='output_segment.mp4'):
    movie_segment = VideoFileClip(movie_path).subclipped(start_time, end_time)
    print(f"\nWriting movie file from {start_time}s until {end_time}s")
    movie_segment.write_videofile(output_path, codec="libx264",
        audio_codec="aac", logger=None)
    return output_path
def display_transcript_and_movie(chunk_index, transcript_df, chunks,
    movie_path):
    start_time, end_time = chunks[chunk_index]
    transcript_chunk = transcript_df.iloc[chunk_index] if chunk_index < len(transcript_df) else None
    print(f"\nChunk number: {chunk_index + 1}")
    if transcript_chunk is not None and pd.notna(transcript_chunk['text_per_tr']):
        print(f"\nText: {transcript_chunk['text_per_tr']}")
        print(f"Words: {transcript_chunk['words_per_tr']}")
        print(f"Onsets: {transcript_chunk.get('onsets_per_tr', 'N/A')}")
        print(f"Durations: {transcript_chunk.get('durations_per_tr', 'N/A')}")
    else:
        print("<No dialogue in this scene>")
    output_movie_path = extract_movie_segment_with_sound(movie_path, start_time,
        end_time)
    display(Video(output_movie_path, embed=True, width=640, height=480))
def create_dropdown_by_text(transcript_df):
    options = []
    for i, row in transcript_df.iterrows():
        if pd.notna(row['text_per_tr']):
            options.append((row['text_per_tr'], i))
        else:
            options.append(("<No dialogue in this scene>", i))
    return widgets.Dropdown(options=options, description='Select scene:')
def plot_fmri_on_brain(chunk_index, fmri_file_path, atlas_path, dataset_name,
    hrf_delay):
    print(f"\nLoading fMRI file: {fmri_file_path}")
    atlas_img = nib.load(atlas_path)
    atlas_data = atlas_img.get_fdata()
    with h5py.File(fmri_file_path, 'r') as f:
        print(f"Opening fMRI dataset: {dataset_name}")
        fmri_data = f[dataset_name][()]
        print(f"fMRI dataset shape: {fmri_data.shape}")
    if (chunk_index + hrf_delay) > len(fmri_data):
        selected_sample = len(fmri_data)
    else:
        selected_sample = chunk_index + hrf_delay
    fmri_sample_data = fmri_data[selected_sample]
    print(f"Extracting fMRI sample {selected_sample+1}.")
    output_data = np.zeros_like(atlas_data)
    for parcel_index in range(1000):
        output_data[atlas_data == (parcel_index + 1)] = \
            fmri_sample_data[parcel_index]
    output_img = nib.Nifti1Image(output_data, affine=atlas_img.affine)
    display = plotting.plot_glass_brain(
        output_img,
        display_mode='lyrz',
        cmap='inferno',
        colorbar=True,
        plot_abs=False)
    colorbar = display._cbar
    colorbar.set_label("fMRI activity", rotation=90, labelpad=12, fontsize=12)
    plotting.show()
def get_fmri_on_brain(chunk_index, fmri_file_path, atlas_path, dataset_name,
    hrf_delay):
    atlas_img = nib.load(atlas_path)
    atlas_data = atlas_img.get_fdata()
    with h5py.File(fmri_file_path, 'r') as f:
        fmri_data = f[dataset_name][()]
    if (chunk_index + hrf_delay) > len(fmri_data):
        selected_sample = len(fmri_data)
    else:
        selected_sample = chunk_index + hrf_delay
    fmri_sample_data = fmri_data[selected_sample]
    output_data = np.zeros_like(atlas_data)
    for parcel_index in range(1000):
        output_data[atlas_data == (parcel_index + 1)] = \
            fmri_sample_data[parcel_index]
    output_img = nib.Nifti1Image(output_data, affine=atlas_img.affine)
    return atlas_img, atlas_data, fmri_data, output_data, output_img
def interface_display_transcript_movie_brain(movie_path, transcript_path,
    fmri_file_path, atlas_path, dataset_name, hrf_delay):
    transcript_df = load_transcript(transcript_path)
    chunks = split_movie_into_chunks(movie_path)
    dropdown = create_dropdown_by_text(transcript_df)
    output = widgets.Output()
    def on_chunk_select(change):
        with output:
            output.clear_output()
            chunk_index = dropdown.value
            display_transcript_and_movie(chunk_index, transcript_df, chunks,
                movie_path)
            plot_fmri_on_brain(chunk_index, fmri_file_path, atlas_path,
                dataset_name, hrf_delay)
    dropdown.observe(on_chunk_select, names='value')
    display(dropdown, output)

In [18]:
# HRF delay parameter
hrf_delay = 0  #@param {type:"slider", min:0, max:10, step:1}

# Define file paths and dataset name
movie_path = root_data_dir + "/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e01a.mkv"
transcript_path = root_data_dir + "/algonauts_2025.competitors/stimuli/transcripts/friends/s1/friends_s01e01a.tsv"
fmri_file_path = root_data_dir + "/algonauts_2025.competitors/fmri/sub-01/func/sub-01_task-friends_space-MNI152NLin2009cAsym_atlas-Schaefer18_parcel-1000Par7Net_desc-s123456_bold.h5"
atlas_path = root_data_dir + "/algonauts_2025.competitors/fmri/sub-01/atlas/sub-01_space-MNI152NLin2009cAsym_atlas-Schaefer18_parcel-1000Par7Net_desc-dseg_parcellation.nii.gz"
dataset_name = "ses-003_task-s01e01a"

# Get the selected transcript row/chunk from the interface
interface_display_transcript_movie_brain(movie_path, transcript_path,
    fmri_file_path, atlas_path, dataset_name, hrf_delay)

Movie FPS: 29.968454258675077, Frame Count: 26412.0


Dropdown(description='Select scene:', options=(('<No dialogue in this scene>', 0), ('<No dialogue in this scen…

Output()

In [ ]:
atlas_img, atlas_data, fmri_data, output_data, output_img = get_fmri_on_brain(129, fmri_file_path, atlas_path, dataset_name, hrf_delay)

In [27]:
algonauts_fmri_viewer = view_img(output_img, title="Algonauts Sorry fMRI View")
algonauts_fmri_viewer.save_as_html("_p_algonauts_fmri_viewer.html")
with open("_p_algonauts_fmri_viewer.html", "r") as file:
    content = file.read()
content = content.replace(
    "<title>Algonauts Sorry fMRI View</title>",
    "<title>_p_algonauts_fmri_viewer.html</title>"
)
with open("_p_algonauts_fmri_viewer.html", "w") as file:
    file.write(content)

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:868: UserWarning:




In [33]:
all_voxels = fmri_data.flatten()
hist, edges = np.histogram(all_voxels, bins=100)
p = figure(
    height=400,
    width=800,
    title="Overall Voxel Intensity Distribution (All Time Points)",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    y_axis_type="log",
    outline_line_color=None,
    min_border_left=50
)
p.yaxis.axis_label = "Frequency (log scale)"
p.xaxis.axis_label = "Voxel Signal Intensity"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_alpha = 0.5
p.ygrid.grid_line_dash = 'dashed'
p.quad(
    top=hist,
    bottom=0.1,
    left=edges[:-1],
    right=edges[1:],
    fill_color="dodgerblue",
    line_color="black",
    alpha=0.8
)
hover = HoverTool(tooltips=[("Range", "@left{0.00} to @right{0.00}"), ("Frequency", "@top")])
p.add_tools(hover)
output_file("_q_algonauts_voxel_value_distribution.html", title="_q_algonauts_voxel_value_distribution.html")
save(p)

'/workspaces/leclei_project/_q_algonauts_voxel_value_distribution.html'

In [37]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([fmri_data])[0]
np.fill_diagonal(correlation_matrix, 0)
fig = plotting.plot_matrix(correlation_matrix, figure=(10, 8), labels=np.arange(1, 1001).tolist(),
                     vmax=0.8, vmin=-0.8, reorder=True)
fig.figure.savefig("_r_algonauts_correlation_matrix.png", dpi=300)
plt.close(fig.figure)